In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
def μg_Iron_per_l_to_μmol_Iron_per_m3(μg_Fe_l):
    """
    Converts μg Iron per liter to μmol Iron per m³
    Arguments:
        μg_Fe_l: float: Iron concentration in μg per l
    Returns
        float: Iron concentration in μmol per m³
    """
    # Convert from μg to μmol
    μmol_Fe_l = μg_Fe_l * 0.0179
    # Convert from l to m³
    μmol_Fe_m3 = μmol_Fe_l * 1000
    return μmol_Fe_m3
    

# Iron limitation in Gracilaria species
https://link.springer.com/article/10.1007/s10811-006-9047-5
